In [5]:
import cv2
import numpy as np
import os

# Function to process each image
def process_image(image_path):
    # Step 1: Read and preprocess the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY)

    # Step 2: Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary,connectivity=4)

    # Step 3: Sort components by area (ignore background, which is stats[0])
    areas = stats[1:, cv2.CC_STAT_AREA]  # Exclude background
    sorted_indices = np.argsort(-areas)  # Sort in descending order

    # Step 4: Get indices of the two largest components
    largest_index = sorted_indices[0] + 1  # +1 to account for exclusion of background
    second_largest_index = sorted_indices[1] + 1

    # Create masks for the two largest components
    largest_component = (labels == largest_index).astype(np.uint8) * 255
    second_largest_component = (labels == second_largest_index).astype(np.uint8) * 255

    # Step 5: Combine both components into one image for visualization
    combined_image = cv2.merge([largest_component, second_largest_component, np.zeros_like(largest_component)])
    combined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)

    # Step 6: Concatenate the original and processed image side by side
    original_image_colored = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convert original to color (BGR)
    concatenated_image = np.hstack((original_image_colored, combined_image))  # Concatenate horizontally

    # Step 7: Resize the image to a smaller size (e.g., 50% of the original size)
    resized_image = cv2.resize(concatenated_image, (0, 0), fx=0.25, fy=0.25)

    # Step 8: Visualize the result in a smaller window
    cv2.imshow(f'Processed {os.path.basename(image_path)}', resized_image)
    cv2.waitKey(0)

# Step 9: Loop through all images in the folder
folder_path = 'data/normal'  # Specify the folder path here
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  # Process only image files
        image_path = os.path.join(folder_path, filename)
        process_image(image_path)

cv2.destroyAllWindows()


KeyboardInterrupt: 